In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.regression_models as RegressionModels
import core.assignment_models as AssignmentModels
import core
import keras

plt.rcParams["font.size"] = 14

PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_from_npz("../../DATA/nominal.npz", max_events=1000000)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)

In [2]:
AssignmentTransformer = AssignmentModels.FeatureConcatTransformer(
    data_config, name=r"Assignment Transformer + $\nu$-Flows"
)
#AssignmentTransformer.load_model(MODEL_DIR + "model.keras")

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


In [3]:
RegressionTransformer = RegressionModels.SimpleNeutrinoRegessor(
    data_config
)
#RegressionTransformer.load_model(MODEL_DIR + "model.keras")

In [4]:
RegressionTransformer.build_model(
    hidden_dim=128,
    num_heads=8,
    central_transformer_stack_size=4,
    neutrino_regression_layers=6,
    dropout_rate=0.1,
)
RegressionTransformer.compile_model(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss = {
        "regression" : core.utils.RegressionLoss(),
        "assignment" : core.utils.AssignmentLoss(),
    },
    metrics={
        "regression" : core.utils.RegressionDeviation(),
        "assignment" : core.utils.AssignmentAccuracy(),
    },
)

RegressionTransformer.train_model(
    X_train,
    y_train,
    batch_size=2048,
    epochs=3,
)


Epoch 1/3
138/352 ━━━━━━━━━━━━━━━━━━━━ 5:06 1s/step - assignment_accuracy: 0.1045 - assignment_loss: 0.0019 - loss: 201.5410 - regression_deviation: 1.1987 - regression_loss: 201.5391

KeyboardInterrupt: 

In [ ]:
RegressionTransformer.save_model(MODEL_DIR + "model.keras")

In [ ]:

import core.assignment_models.BaselineAssignmentMethods as BaselineMethods
import core.reconstruction as Evaluation
reload(Evaluation)
reload(BaselineMethods)
delta_r_assigner = BaselineMethods.DeltaRAssigner(data_config)
chi_square_true_nu = BaselineMethods.ChiSquareAssigner(
    data_config,
    top_mass=173.5e3,
)
ground_truth_assigner = Evaluation.GroundTruthReconstructor(
    data_config, name="Perfect Reconstructor"
)
evaluator = Evaluation.ReconstructionEvaluator(
    [
        chi_square_true_nu,
        RegressionTransformer,
        AssignmentTransformer,
        ground_truth_assigner,
    ],
    X_val,
    y_val,
)

In [ ]:
fig, ax = evaluator.plot_all_accuracies()
fig.savefig(PLOTS_DIR + "all_accuracies.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_top_mass_resolution(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(PLOTS_DIR + "top_mass_resolution_comparison.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_ttbar_mass_resolution(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)

fig.savefig(PLOTS_DIR + "ttbar_mass_resolution_comparison.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_complementarity(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)

fig.savefig(PLOTS_DIR + "binned_complementarity_ttbar_mass.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
    bins=10,
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(PLOTS_DIR + "binned_accuracy_ttbar_mass.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="truth_ttbar_pt",
    fancy_feature_label=r"$p_T(t\overline{t})$ [GeV]",
    xlims=(0, 400e3),
    bins=10,
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(0, 400e3)

fig.savefig(PLOTS_DIR + "binned_accuracy_ttbar_pT.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="truth_initial_parton_num_gluons",
    fancy_feature_label=r"Initial State",
    xlims=(0, 3),
    bins=3,
)
ax.set_xticks([0.5, 1.5, 2.5])
ax.set_xticklabels([r"$qq\to tt$", r"$qg\to tt$", r"$gg\to tt$"])
fig.savefig(PLOTS_DIR + "binned_accuracy_initial_state.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="N_jets",
    fancy_feature_label=r"$\# \text{jets}$",
    xlims=(2, data_configs["LoadConfig"]["max_jets"] + 1),
    bins=data_configs["LoadConfig"]["max_jets"] - 1,
)
ax.set_xticks([i + 0.5 for i in range(2, data_configs["LoadConfig"]["max_jets"] + 1)])
ax.set_xticklabels([i for i in range(2, data_configs["LoadConfig"]["max_jets"] + 1)])
fig.savefig(PLOTS_DIR + "binned_accuracy_N_jets.pdf")

In [ ]:
fig, ax = evaluator.plot_confusion_matrices()
fig.savefig(PLOTS_DIR + "confusion_matrices.pdf")

In [ ]:
# Plot neutrino component deviation histograms
fig, axes = evaluator.plot_neutrino_component_deviations(
    bins=50,
    xlims=(0, 5),  # Optional: limit x-axis range
    figsize=(20, 15),
    component_labels=["$p_x$", "$p_y$", "$p_z$"],
)

In [ ]:
# Plot overall relative deviation distribution
fig, ax = evaluator.plot_all_neutrino_deviations()